Use Autoencoder to implement anomaly detection. Build the model by using:
a. Import required libraries
b. Upload / access the dataset
c. Encoder converts it into latent representation
d. Decoder networks convert it back to the original input
e. Compile the models with Optimizer, Loss, and Evaluation Metrics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler


In [ ]:
data = pd.read_csv(r"C:\Users\ASUS\Downloads\creditcard.csv")
print(data.info())
print(data.head())


In [ ]:
# Separate Features and Labels
X = data.drop(columns=['Class'])
y = data['Class']


In [ ]:
# Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
# Separate Normal and Anomalous Transactions
X_normal = X_scaled[y == 0]
X_anomalous = X_scaled[y == 1]


In [ ]:
# Build the Autoencoder Model
input_dim = X_normal.shape[1]  # Input dimension based on features in the dataset


In [ ]:
# Encoder
input_layer = Input(shape=(input_dim,))
encoder = Dense(64, activation="relu")(input_layer)
encoder = Dense(32, activation="relu")(encoder)
latent_space = Dense(16, activation="relu")(encoder)

# Decoder
decoder = Dense(32, activation="relu")(latent_space)
decoder = Dense(64, activation="relu")(decoder)
output_layer = Dense(input_dim, activation="sigmoid")(decoder)

# Build and Compile the Autoencoder Model
autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [ ]:
# Train the Autoencoder on Normal Transactions
history = autoencoder.fit(X_normal, X_normal, 
                          epochs=10, 
                          batch_size=32, 
                          validation_split=0.2, 
                          shuffle=True)

In [ ]:
# Set Anomaly Detection Threshold
X_normal_pred = autoencoder.predict(X_normal)
reconstruction_error_normal = np.mean(np.square(X_normal_pred - X_normal), axis=1)
threshold = np.percentile(reconstruction_error_normal, 95)
print("Anomaly detection threshold:", threshold)


In [ ]:
# Detect Anomalies in Anomalous Data
X_anomalous_pred = autoencoder.predict(X_anomalous)
reconstruction_error_anomalous = np.mean(np.square(X_anomalous_pred - X_anomalous), axis=1)
anomalies = reconstruction_error_anomalous > threshold
print("Number of anomalies detected:", np.sum(anomalies))

In [ ]:
# Visualize Reconstruction Error
plt.hist(reconstruction_error_normal, bins=50, alpha=0.7, label="Normal")
plt.hist(reconstruction_error_anomalous, bins=50, alpha=0.7, label="Anomaly")
plt.axvline(threshold, color='red', linestyle='--', label="Threshold")
plt.xlabel("Reconstruction error")
plt.ylabel("Frequency")
plt.legend()
plt.show()